In [7]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import CRUD module
import nbimporter
from animal_shelter_test import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser2"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password, 'nv-desktop-services.apporto.com', 32490, 'AAC', 'animals')

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Grazioso Salvare’s logo
image_filename = './dashboard_assets/Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# dashboard layout
app.layout = html.Div([
    # html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Hr(),
    html.Div(
        # Interactive filtering options
        dcc.RadioItems(
            id='animal-radio',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster Rescue', 'value': 'Disaster Rescue'},
                {'label': 'Reset', 'value': 'All'},
            ],
            value='All'  # Default value
        ),
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",  # Allow single row selection
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ]),
    # unique identifier
    html.H5('Dashboard created by Sean Glass.')
])

#############################################
# Interaction Between Components / Controller
#############################################


@app.callback(
    [
        Output('datatable-id', 'data'),
        Output('map-id', 'children')
    ],
    [
        Input('animal-radio', 'value'),
        Input('datatable-id', 'derived_virtual_data'),
        Input('datatable-id', 'derived_virtual_selected_rows')
    ]
)
def update_dashboard_and_map(selected_filter, viewData, index):
    # Initialize variables for output
    updated_data = []
    map_children = []

    ## Filter interactive data table with MongoDB queries
    if selected_filter == 'All':
        updated_data = df.to_dict('records')
    elif selected_filter == 'Water Rescue':
        # Filter based on Water Rescue criteria
        filtered_df = df[
            (df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])) &
            (df['sex_upon_outcome'] == 'Intact Female') &
            (df['age_upon_outcome_in_weeks']).between(26.0, 156.0)
        ]
        updated_data = filtered_df.to_dict('records')
    elif selected_filter == 'Mountain or Wilderness Rescue':
        # Filter based on Mountain or Wilderness Rescue criteria
        filtered_df = df[
            (df['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 
                              'Siberian Husky', 'Rottweiler'])) &
            (df['sex_upon_outcome'] == 'Intact Male') &
            (df['age_upon_outcome_in_weeks']).between(26.0, 156.0)
        ]
        updated_data = filtered_df.to_dict('records')
    elif selected_filter == 'Disaster Rescue':
        # Filter based on Disaster Rescue criteria
        filtered_df = df[
            (df['breed'].isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 
                              'Rottweiler'])) &
            (df['sex_upon_outcome'] == 'Intact Male') &
            (df['age_upon_outcome_in_weeks']).between(26.0, 156.0)
        ]
        updated_data = filtered_df.to_dict('records')

    # Check for available data
    if viewData is not None and len(viewData) > 0:
        dff = pd.DataFrame.from_dict(viewData)
        # Because we only allow single row selection, the list can be converted to a row index here
        if index is not None and len(index) > 0 and index[0] < len(dff):
            selected_row_index = index[0]
            # Ensure that the required columns are not empty or contain valid data
            if pd.notna(dff.iloc[selected_row_index, 13]) and pd.notna(dff.iloc[selected_row_index, 14]):
                lat = dff.iloc[selected_row_index, 13]
                lon = dff.iloc[selected_row_index, 14]
                animal_name = dff.iloc[selected_row_index, 9]

                map_children = [
                    dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
                        dl.TileLayer(id="base-layer-id"),
                        # Marker with popup
                        dl.Marker(position=[lat, lon], children=[
                            dl.Popup([
                                html.H1("Animal Name"),
                                html.P(animal_name),
                            ])
                        ])
                    ])
                ]

    return updated_data, map_children


# This callback will display the pie chart based on the filtered data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('animal-radio', 'value')]
)
def update_graphs(viewData, selected_filter):
    # Adds pie chart
    if viewData is None or len(viewData) == 0:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Check if a filter is applied
    if selected_filter != 'All' and len(dff) > 0 and len(dff['breed'].unique()) > 0:
        # If filter is applied and multiple breeds are present, show 'breed'
        pie = px.pie(dff, names='breed')
    else:
        # If no filter is applied or only one breed is present, show 'animal_type'
        pie = px.pie(dff, names='animal_type')
    
    return dcc.Graph(figure=pie)


# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns: # handle case when no columns are selected
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


app.run_server(debug=True)


Dash app running on http://127.0.0.1:12643/
